In [1]:
import pandas as pd
import numpy as np
from bff_processor.data_tools import regex_select, get_files, make_df
from bff_processor.bff_meta import preselection, band_cut, isin, identity, all_reg, jet_variations
from bff_processor.bff_cuts import *
from glob import glob
import re

In [9]:
era = 2017
regex_select = ".+/tw_{}_(.+)\.csv".format(era)
files = [x for x in glob('data/*') if re.match(regex_select, x)]
DY = [x for x in files if re.match('.+ZTo(?:Mu|EE).+', x)]
ST = [x for x in files if re.match('.+top.csv', x)]
VB = [x for x in files if re.match('.+mc_(?:ww|wz|zz)', x)]
TT = [x for x in files if re.match('.+ttbar', x)]
BFF = [x for x in files if re.match('.+BFFZp', x)]
data = [x for x in files if re.match('.+_data_', x)]
assert len(files) == len(DY+ST+TT+VB+data+BFF), "duplicate or uncaught file"

In [15]:
sample_types = [
['.+ZTo(?:Mu|EE).+', 'bck'],
['.+top.csv', 'bck'],
['.+mc_(?:ww|wz|zz)', 'bck'],
['.+ttbar', 'bck'],
['.+BFFZp', 'sig'],
['.+_data_', 'data']
]

In [16]:
def process_file(file):
    name = re.findall(regex_select, file)[0]
    #get stuff for bff samples
    dbs = re.findall('dbs(\dp\d+)', name)
    mass = re.findall('_M_(\d+)', name)
    if len(dbs) == 1 : dbs = float(dbs[0].replace('p','.'))
    else: dbs = None
    if len(mass) == 1 : mass = int(mass[0])
    else: mass = None
    type_label = 0
    for re_sample, type_sample in sample_types:
        if re.match(re_sample, file): 
            type_label = type_sample
            break
            
    #open file and filter out events with bff selection
    df = preselection()(pd.read_csv(file))
    
    for reg in df.filter(regex='(?:SR|CR)\d+_.+'):
        nJets, jv = re.findall('(?:SR|CR)(\d)\d*_(.+)', reg)[0]
        df[reg] = df[reg]*bff_no_tmb_value[nJets](df, jv)
    selected_events = df.filter(regex='(?:SR|CR)\d+_.+').sum(axis=1)>0
    print( "{} remaining".format(   (df.filter(regex='(?:SR|CR)\d+_.+').sum(axis=1)>0).mean()))
    df = df[selected_events]
    #remove unnamed column from index, probably a better way, but ok for now
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    
    #add metadata
    df['name'] = name
    df['dbs'] = dbs
    df['mass'] = mass
    df['type'] = type_label
    return df


In [17]:
combined_df = pd.DataFrame()
for file in files:
    print(file)
    df = process_file(file)
    combined_df = pd.concat([combined_df, df])

data/tw_2018_BFFZprimeToMuMu_M_300_dbs0p04.csv
0.8635057681877483 remaining
data/tw_2018_mc_wz.csv
0.6926713947990544 remaining
data/tw_2018_ZToMuMu_M_50_120.csv
0.9622641509433962 remaining
data/tw_2018_ZToMuMu_M_800_1400.csv
0.2511467889908257 remaining
data/tw_2018_data_mu.csv
0.6385354739270704 remaining
data/tw_2018_mc_zz.csv
0.7973421926910299 remaining
data/tw_2018_ZToMuMu_M_200_400.csv
0.8972809667673716 remaining
data/tw_2018_BFFZprimeToMuMu_M_175_dbs0p04.csv
0.9338082517067379 remaining
data/tw_2018_mc_ww.csv
0.5758738862234407 remaining
data/tw_2018_ZToEE_M_200_400.csv
0.8854033918243309 remaining
data/tw_2018_ZToEE_M_800_1400.csv
0.35671191553544496 remaining
data/tw_2018_BFFZprimeToMuMu_M_500_dbs0p04.csv
0.58553156959221 remaining
data/tw_2018_mc_ttbar.csv
0.4199483555313944 remaining
data/tw_2018_BFFZprimeToMuMu_M_200_dbs1p0.csv
0.9297859384514497 remaining
data/tw_2018_ZToEE_M_50_120.csv
0.9549071618037135 remaining
data/tw_2018_BFFZprimeToMuMu_M_350_dbs0p5.csv
0.8074347

In [18]:
import pyarrow.feather as feather
feather.write_feather(combined_df, 'data/combined_{}.feather'.format(era))